In [1]:
import sys, os
import numpy as np
np.set_printoptions(threshold=np.inf)
import pandas as pd
from utils.utils import read_txt, read_txt_np_int
from functools import lru_cache
from math import ceil, floor
from copy import deepcopy

# INPUT

In [2]:
inputfilename = './inputs/day21.txt'

inputdata = read_txt(inputfilename)

testdata_small = ["029A",
"980A",
"179A",
"456A",
"379A"]

code_keyboard = np.array([['7', '8', '9'],['4', '5', '6'],['1', '2', '3'],['', '0', 'A']])

direction_keyboard = np.array([['', '^', 'A'],['<', 'v', '>']])

In [36]:
data_to_use = inputdata

movement_map = {'<': np.array([0, -1]), 'v': np.array([1, 0]), '>': np.array([0, 1]), '^': np.array([-1, 0])}

keyboard_chain = ['movement', 'movement', 'movement', 'code']

data_to_use

['638A', '965A', '780A', '803A', '246A']

## PART 1

In [37]:
class Robot:

    def __init__(self, robot_dict, keyboard):
        self.dict = robot_dict
        self.keyboard = keyboard
        self.sign = 'A'

    # Get the list of transitions from a code
    @lru_cache()
    def process_code_reverse(self, code, sign = 'A'):
        current_sign = sign
        transition_movements = []
        next_sign = code[0]
        transition = current_sign + next_sign
        transition_movements.append(self.dict[transition])
        if len(code) <= 1:
            return transition_movements
        transition_movements += self.process_code_reverse(code[1:], next_sign)

        # Compact list
        pending_movement = ''
        output_movements = []

        for movement_list in transition_movements:
            if len(movement_list) == 1:
                pending_movement += movement_list[0]
                continue
            if len(pending_movement) > 0:
                output_movements.append([pending_movement])
            output_movements.append(movement_list)
            pending_movement = ''
        
        if len(pending_movement) > 0:
            output_movements.append([pending_movement])

        return output_movements
    
    def process_code(self, code, sign = 'A'):
        position = (int(np.where(self.keyboard == sign)[0][0]), int(np.where(self.keyboard == sign)[1][0]))
        output_values = []
        for char in code:
            if char == 'A':
                output_values += self.keyboard[position[0], position[1]]
            else:
                movement = movement_map[char]
                position = (position[0]+movement[0], position[1]+movement[1])
        return ''.join(output_values)


def all_orderings(movement_string):
    if len(movement_string) <= 1:
        return set(movement_string)
    orderings = set()
    for index in range(len(movement_string)):
        for next_ordering in all_orderings(movement_string[:index] + movement_string[index+1:]):
            orderings.add(movement_string[index] + next_ordering)
    return orderings

def get_all_paths(starting_position, final_position):
    vertical_delta = final_position[0] - starting_position[0]
    horizontal_delta = final_position[1] - starting_position[1]
    movement_string = ''
    vertical_sign, horizontal_sign = '', ''
    if vertical_delta > 0:
        vertical_sign = 'v'
    if vertical_delta < 0:
        vertical_sign = '^'
    if horizontal_delta > 0:
        horizontal_sign = '>'
    if horizontal_delta < 0:
        horizontal_sign = '<'

    movement_string += abs(vertical_delta)*vertical_sign + abs(horizontal_delta)*horizontal_sign

    # Each transition has to finish by pressing A
    all_movements = [movement + 'A' for movement in all_orderings(movement_string)]

    if horizontal_delta == vertical_delta == 0:
        all_movements = ['A']

    return all_movements

def valid_path(keyboard, starting_position, path):
    current_position = starting_position
    for direction in path:
        if direction == 'A':
            continue
        movement = movement_map[direction]
        current_position = (current_position[0] + movement[0], current_position[1]+movement[1])
        # Panic check
        #print(f"Visited position: {current_position}")
        if keyboard[current_position[0],current_position[1]] == '':
            return False
    return True

def process_keyboard(keyboard):
    keyboard_dict = {}
    for starting_sign in keyboard.flatten():
        if starting_sign == '':
            continue
        for ending_sign in keyboard.flatten():
            if ending_sign == '':
                continue

            # Only transition to a different sign (why?)
            if ending_sign == starting_sign:
                pass        # let's keep all transitions
            starting_position = (int(np.where(keyboard == starting_sign)[0][0]), int(np.where(keyboard == starting_sign)[1][0]))
            ending_position = (int(np.where(keyboard == ending_sign)[0][0]), int(np.where(keyboard == ending_sign)[1][0]))
            transition = starting_sign + ending_sign

            tentative_paths = get_all_paths(starting_position, ending_position)
            keyboard_dict[transition] = [path for path in tentative_paths if valid_path(keyboard, starting_position, path)]
    return keyboard_dict

# Each combination to be entered requires an 'A' after each symbol -- only for movement
def expand_code_list(codelist, minimize = False):
    expanded_code_list = ['']
    for codeset in codelist:
        next_expanded_code_list = []
        if minimize:
            min_len = 1000000
            for code in codeset:
                if len(code) < min_len:
                    shortest_code = code
            expanded_code_list = [expanded_code_list[0] + shortest_code]
        else:
            for code in codeset:
                for previous_code in expanded_code_list:
                    next_expanded_code_list.append(previous_code + code)
            expanded_code_list = next_expanded_code_list
    return expanded_code_list

'''
def process_code(code, keyboard_dict, starting_sign):
    current_sign = starting_sign
    transition_movements = []
    for next_sign in code:
        transition = current_sign + next_sign
        #print(transition)
        current_sign = next_sign
        transition_movements.append(keyboard_dict[transition])
    return transition_movements
'''


######################################################################################################################

direction_keyboard_dict = process_keyboard(direction_keyboard)
code_keyboard_dict = process_keyboard(code_keyboard)

keyboard_config = {'movement': direction_keyboard_dict, 'code': code_keyboard_dict}

robot_code = Robot(keyboard_config['code'], code_keyboard) # Returns input for previous

robot_movement_1 = Robot(keyboard_config['movement'], direction_keyboard) # Returns input for previous
#robot_movement_2 = Robot(keyboard_config['movement'], direction_keyboard) # Returns input for previous


In [5]:
keyboard_config['movement']

{'^^': ['A'],
 '^A': ['>A'],
 '^<': ['v<A'],
 '^v': ['vA'],
 '^>': ['v>A', '>vA'],
 'A^': ['<A'],
 'AA': ['A'],
 'A<': ['v<<A', '<v<A'],
 'Av': ['v<A', '<vA'],
 'A>': ['vA'],
 '<^': ['>^A'],
 '<A': ['>>^A', '>^>A'],
 '<<': ['A'],
 '<v': ['>A'],
 '<>': ['>>A'],
 'v^': ['^A'],
 'vA': ['>^A', '^>A'],
 'v<': ['<A'],
 'vv': ['A'],
 'v>': ['>A'],
 '>^': ['<^A', '^<A'],
 '>A': ['^A'],
 '><': ['<<A'],
 '>v': ['<A'],
 '>>': ['A']}

In [38]:
cumulator = 0


for entry_code in data_to_use:

    print(entry_code)

    starting_symbol = 'A'

    code_list = [set([entry_code])]
    expanded_code_list = expand_code_list(code_list)

    keyboard = 'code'

    print(f"Processing keyboard {keyboard}")
    print(f"Expanded code list: {expanded_code_list}")
    previous_code_list = []
    for code in expanded_code_list:
        print(code)
        print(robot_code.process_code_reverse(code, starting_symbol))
        previous_code_list += expand_code_list(robot_code.process_code_reverse(code))

    print(f"Previous code list: {previous_code_list}") # movement 1

    #######################################################################################################################

    expanded_code_list = set(previous_code_list)
    keyboard = 'movement'

    print(f"Processing keyboard {keyboard}")
    print(f"Expanded code list: {expanded_code_list}")
    previous_code_list = []
    for code in expanded_code_list:
        previous_code_list += expand_code_list(robot_movement_1.process_code_reverse(code))

    print(f"Previous code list: {previous_code_list}") # movement 2

    #######################################################################################################################

    expanded_code_list = set(previous_code_list)
    keyboard = 'movement'

    print(f"Processing keyboard {keyboard}")
    print(f"Expanded code list: {expanded_code_list}")
    previous_code_list = []
    for code in expanded_code_list:
        previous_code_list += expand_code_list(robot_movement_1.process_code_reverse(code), minimize = False)

    #print(f"Previous code list: {previous_code_list}") # movement 3

    min_len = 1000000
    for code in previous_code_list:
        if len(code) < min_len:
            min_len = len(code)
            shortest = code

    print(f"Shortest code: {shortest}, len {min_len}")

    cumulator += min_len * int(entry_code[:-1])

print(cumulator)


638A
Processing keyboard code
Expanded code list: ['638A']
638A
[['^^AvA'], ['<^^A', '^<^A', '^^<A'], ['v>vvA', 'vvv>A', 'vv>vA', '>vvvA']]
Previous code list: ['^^AvA<^^Av>vvA', '^^AvA^<^Av>vvA', '^^AvA^^<Av>vvA', '^^AvA<^^Avvv>A', '^^AvA^<^Avvv>A', '^^AvA^^<Avvv>A', '^^AvA<^^Avv>vA', '^^AvA^<^Avv>vA', '^^AvA^^<Avv>vA', '^^AvA<^^A>vvvA', '^^AvA^<^A>vvvA', '^^AvA^^<A>vvvA']
Processing keyboard movement
Expanded code list: {'^^AvA^^<Avv>vA', '^^AvA^<^Avvv>A', '^^AvA^<^Av>vvA', '^^AvA^^<Av>vvA', '^^AvA<^^Avvv>A', '^^AvA<^^A>vvvA', '^^AvA<^^Av>vvA', '^^AvA<^^Avv>vA', '^^AvA^^<Avvv>A', '^^AvA^<^Avv>vA', '^^AvA^<^A>vvvA', '^^AvA^^<A>vvvA'}
Previous code list: ['<AA>Av<A>^A<AAv<A>>^Av<AA>A<A>^A', '<AA>A<vA>^A<AAv<A>>^Av<AA>A<A>^A', '<AA>Av<A^>A<AAv<A>>^Av<AA>A<A>^A', '<AA>A<vA^>A<AAv<A>>^Av<AA>A<A>^A', '<AA>Av<A>^A<AAv<A>^>Av<AA>A<A>^A', '<AA>A<vA>^A<AAv<A>^>Av<AA>A<A>^A', '<AA>Av<A^>A<AAv<A>^>Av<AA>A<A>^A', '<AA>A<vA^>A<AAv<A>^>Av<AA>A<A>^A', '<AA>Av<A>^A<AAv<A>>^A<vAA>A<A>^A', '<AA>A<vA>^A

In [39]:
command_to_test = '<vA<AA^>>AA<A>vA^AvA^A<<vA^>>AAvA^A<vA>^AA<A>A<<vA>A>^AAA<A>vA^A'
print(f"Command lenght {len(command_to_test)}")
first_output = robot_movement_2.process_code(command_to_test, 'A')
print(first_output)
second_output = robot_movement_1.process_code(first_output, 'A')
print(second_output)
final_output = robot_code.process_code(second_output, 'A')
print(final_output)

Command lenght 64


NameError: name 'robot_movement_2' is not defined

## PART 2

### First try

In [16]:
class Robot_Lite:

    def __init__(self, robot_dict, keyboard):
        self.dict = robot_dict
        self.keyboard = keyboard
        self.sign = 'A'

    # Get the list of transitions from a code
    @lru_cache()
    def process_code_reverse(self, code, sign = 'A'):
        current_sign = sign
        transition_movements = []
        next_sign = code[0]
        
        for next_sign in code:
            transition = current_sign + next_sign
            transition_movements.append(self.dict[transition])

        # Compact list
        pending_movement = ''
        output_movements = []
        for movement_list in transition_movements:
            if len(movement_list) == 1:
                pending_movement += movement_list[0]
                continue
            if len(pending_movement) > 0:
                output_movements.append([pending_movement])
            output_movements.append(movement_list)
            pending_movement = ''
        
        if len(pending_movement) > 0:
            output_movements.append([pending_movement])

        return output_movements
    
    def process_code(self, code, sign = 'A'):
        position = (int(np.where(self.keyboard == sign)[0][0]), int(np.where(self.keyboard == sign)[1][0]))
        output_values = []
        for char in code:
            if char == 'A':
                output_values += self.keyboard[position[0], position[1]]
            else:
                movement = movement_map[char]
                position = (position[0]+movement[0], position[1]+movement[1])
        return ''.join(output_values)


def send_code(code, num_dir_robots):
    for next in range(num_dir_robots):
        code = robot_movement_1.process_code(code, 'A')
    code = robot_code.process_code(code, 'A')
    print(code)

robot_code_lite = Robot_Lite(keyboard_config['code'], code_keyboard) # Returns input for previous

robot_movement_lite = Robot_Lite(keyboard_config['movement'], direction_keyboard) # Returns input for previous


In [11]:
send_code('<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A', 2)

029A


In [12]:
robot_movement_1.process_code_reverse('<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A')

[['<v<A', 'v<<A'],
 ['>A'],
 ['>^A', '^>A'],
 ['<v<A', 'v<<A'],
 ['>>^A', '>^>A'],
 ['AvAA'],
 ['^<A', '<^A'],
 ['>A'],
 ['v<A', '<vA'],
 ['>^A', '^>A'],
 ['A'],
 ['<v<A', 'v<<A'],
 ['>^A>AvA^A'],
 ['<v<A', 'v<<A'],
 ['>A<A'],
 ['>>^A', '>^>A'],
 ['vAA'],
 ['^<A', '<^A'],
 ['>A'],
 ['v<A', '<vA'],
 ['>^A', '^>A'],
 ['<A>A'],
 ['<v<A', 'v<<A'],
 ['>A'],
 ['>^A', '^>A'],
 ['vA'],
 ['^<A', '<^A'],
 ['>A'],
 ['<v<A', 'v<<A'],
 ['>A<A'],
 ['>>^A', '>^>A'],
 ['vA'],
 ['^<A', '<^A'],
 ['>AvA^AA'],
 ['v<A', '<vA'],
 ['>^A', '^>A'],
 ['<A>A'],
 ['<v<A', 'v<<A'],
 ['>A<A'],
 ['>>^A', '>^>A'],
 ['vA^AvA'],
 ['^<A', '<^A'],
 ['>AAA'],
 ['v<A', '<vA'],
 ['>^A', '^>A'],
 ['<v<A', 'v<<A'],
 ['>^A>AvA^A']]

In [17]:
cumulator = 0
num_movement_robots = 25

for entry_code in data_to_use:

    print(entry_code)

    starting_symbol = 'A'

    code_list = [set([entry_code])]
    expanded_code_list = expand_code_list(code_list)

    keyboard = 'code'

    print(f"Processing keyboard {keyboard}")
    print(f"Expanded code list: {expanded_code_list}")
    previous_code_list = []
    for code in expanded_code_list:
        print(code)
        previous_code_list += expand_code_list(robot_code_lite.process_code_reverse(code))

    print(f"Previous code list: {previous_code_list}") # movement 1

    #######################################################################################################################

    for movement_robot_index in range(num_movement_robots):

        print(f"Movement robot depth {movement_robot_index}")
        expanded_code_list = set(previous_code_list)
        keyboard = 'movement'

        previous_code_list = []
        for code in expanded_code_list:
            previous_code_list += expand_code_list(robot_movement_lite.process_code_reverse(code), minimize=True)

    min_len = 1000000
    for code in previous_code_list:
        if len(code) < min_len:
            min_len = len(code)
            shortest = code

    print(f"Shortest code: {shortest}, len {min_len}")
    cumulator += min_len * int(entry_code[:-1])

print(cumulator)


029A
Processing keyboard code
Expanded code list: ['029A']
029A
Previous code list: ['<A^<A^^^AA', '<A<^A^^^AA']
Movement robot depth 0
Movement robot depth 1
Movement robot depth 2
Movement robot depth 3
Movement robot depth 4
Movement robot depth 5
Movement robot depth 6
Movement robot depth 7
Movement robot depth 8
Movement robot depth 9
Movement robot depth 10
Movement robot depth 11
Movement robot depth 12


KeyboardInterrupt: 

### Second try

In [53]:
# Create idem-mapping

# We will pick any set with repeated consecutive signs - minimum shift count
def pick_preferred(set):
    print(f'\t\tselecting preferred from {set}')
    shift_key = {}
    for code in set:
        shifts = sum([code[i] != code[i+1] for i in range(len(code)-1)])
        if shifts in shift_key:
            shift_key[shifts].append(code)
        else:
            shift_key[shifts] = [code]

    return shift_key[min(shift_key.keys())]

starting_set = ['v', '^', '<', '>', 'A']

pending_set = set(starting_set.copy())

idem_mapping = {}

while len(pending_set) > 0:
    next_set = pending_set.pop()
    print(f"Processing set: {next_set}")
    if next_set in idem_mapping.keys():
        continue

    previous_set_list = expand_code_list(robot_movement_1.process_code_reverse(next_set))
    print(f"Generated previous set list: {previous_set_list}")
    
    preferred = pick_preferred(previous_set_list)

    idem_mapping[next_set] = preferred
    print(preferred)

    # Idemrelations end with A -- we break by A and process the bits
    for previous_set in previous_set_list:
        sets_to_review = previous_set.replace('A', 'A#').rstrip('#').split('#')
        for set_to_review in sets_to_review:
            pending_set.add(set_to_review)

idem_mapping

Processing set: ^
Generated previous set list: ['<A']
		selecting preferred from ['<A']
['<A']
Processing set: <
Generated previous set list: ['v<<A', '<v<A']
		selecting preferred from ['v<<A', '<v<A']
['v<<A']
Processing set: <v<A
Generated previous set list: ['v<<A>A<A>>^A', '<v<A>A<A>>^A', 'v<<A>A<A>^>A', '<v<A>A<A>^>A']
		selecting preferred from ['v<<A>A<A>>^A', '<v<A>A<A>>^A', 'v<<A>A<A>^>A', '<v<A>A<A>^>A']
['v<<A>A<A>>^A']
Processing set: >^>A
Generated previous set list: ['vA<^Av>A^A', 'vA^<Av>A^A', 'vA<^A>vA^A', 'vA^<A>vA^A']
		selecting preferred from ['vA<^Av>A^A', 'vA^<Av>A^A', 'vA<^A>vA^A', 'vA^<A>vA^A']
['vA<^Av>A^A', 'vA^<Av>A^A', 'vA<^A>vA^A', 'vA^<A>vA^A']
Processing set: v<<A
Generated previous set list: ['v<A<AA>>^A', '<vA<AA>>^A', 'v<A<AA>^>A', '<vA<AA>^>A']
		selecting preferred from ['v<A<AA>>^A', '<vA<AA>>^A', 'v<A<AA>^>A', '<vA<AA>^>A']
['v<A<AA>>^A', '<vA<AA>>^A']
Processing set: A
Generated previous set list: ['A']
		selecting preferred from ['A']
['A']
Proc

{'^': ['<A'],
 '<': ['v<<A'],
 '<v<A': ['v<<A>A<A>>^A'],
 '>^>A': ['vA<^Av>A^A', 'vA^<Av>A^A', 'vA<^A>vA^A', 'vA^<A>vA^A'],
 'v<<A': ['v<A<AA>>^A', '<vA<AA>>^A'],
 'A': ['A'],
 '>': ['vA'],
 'v': ['v<A', '<vA'],
 'v<A': ['v<A<A>>^A', '<vA<A>>^A'],
 'v>A': ['v<A>A^A', '<vA>A^A'],
 '<vA': ['v<<A>A>^A', 'v<<A>A^>A'],
 'vA': ['v<A>^A', '<vA>^A', 'v<A^>A', '<vA^>A'],
 '>>^A': ['vAA<^A>A', 'vAA^<A>A'],
 '<A': ['v<<A>>^A'],
 '>^A': ['vA<^A>A', 'vA^<A>A'],
 '^<A': ['<Av<A>>^A'],
 '>vA': ['vA<A>^A', 'vA<A^>A'],
 '^>A': ['<Av>A^A', '<A>vA^A'],
 '>A': ['vA^A'],
 '^A': ['<A>A'],
 '<^A': ['v<<A>^A>A']}

In [42]:
pick_preferred(['v<A', '<vA'])

		selecting preferred from ['v<A', '<vA']


'v<A'

In [54]:
@lru_cache
def required_operations_idem_part(code, depth):
    if depth == 0:
        return(len(code))
    depth -= 1
    possible_values = []
    for possibility in idem_mapping[code]:
        code_bits = possibility.replace('A', 'A#').rstrip('#').split('#')
        possible_values.append(sum([required_operations_idem_part(code_bit, depth) for code_bit in code_bits]))
    return(min(possible_values))

In [55]:
# Try WITH LRU CACHE
# Prepare replace mappings
digit_to_number_mapping = {}
number_to_digit_mapping = {}
count = 0
for key in idem_mapping:
    if len(key) == 1:
        number_to_digit_mapping[str(count)] = key
        digit_to_number_mapping[key] = str(count)
        count += 1

print(number_to_digit_mapping)
print(digit_to_number_mapping)

def trackback_code(code, num_robots):

    print(f"Evaluating code: {code}")
    code_list = [set([code])]
    expanded_code_list = expand_code_list(code_list)


    # 1. Convert the code to movement
    keyboard = 'code'
    print(f"Processing keyboard {keyboard}")
    print(f"Expanded code list: {expanded_code_list}")
    previous_code_list = []
    for code in expanded_code_list:
        previous_code_list += expand_code_list(robot_code.process_code_reverse(code))
    
    # Let's optimize and keep only the preferred option (minimum number of symbol shifts)
    #previous_code_list = [pick_preferred(previous_code_list)]

    print(f"\tPrevious code list: {previous_code_list}") # movement 1

    # 2. Next movement step is special because it outputs to full code robot
    intermediate_code_list = []
    for code_movement in previous_code_list:

        intermediate_code_list += expand_code_list(robot_movement_1.process_code_reverse(code_movement))

    print(f"\t\tIntermediate code list: {intermediate_code_list}")
    codelengths = [len(intercode) for intercode in intermediate_code_list]
    print(f"\t\tLength per code: {codelengths}")

    # 3. Remaining use idem-mapping
    effort_list = []
    for code_movement in intermediate_code_list:
        effort = sum([required_operations_idem_part(code_bit, num_robots - 1) for code_bit in code_movement.replace('A', 'A#').rstrip('#').split('#')])
        effort_list.append(effort)
               
    min_effort = min(effort_list)
    return(min_effort, int(code[:-1])*min_effort)

total_cost = 0
for entry_code in data_to_use:
    
    effort, cost = trackback_code(entry_code, 25)
    total_cost += cost

print(total_cost)


{'0': '^', '1': '<', '2': 'A', '3': '>', '4': 'v'}
{'^': '0', '<': '1', 'A': '2', '>': '3', 'v': '4'}
Evaluating code: 638A
Processing keyboard code
Expanded code list: ['638A']
	Previous code list: ['^^AvA<^^Av>vvA', '^^AvA^<^Av>vvA', '^^AvA^^<Av>vvA', '^^AvA<^^Avvv>A', '^^AvA^<^Avvv>A', '^^AvA^^<Avvv>A', '^^AvA<^^Avv>vA', '^^AvA^<^Avv>vA', '^^AvA^^<Avv>vA', '^^AvA<^^A>vvvA', '^^AvA^<^A>vvvA', '^^AvA^^<A>vvvA']
		Intermediate code list: ['<AA>Av<A>^Av<<A>^AA>Av<A>A<AA>^A', '<AA>A<vA>^Av<<A>^AA>Av<A>A<AA>^A', '<AA>Av<A^>Av<<A>^AA>Av<A>A<AA>^A', '<AA>A<vA^>Av<<A>^AA>Av<A>A<AA>^A', '<AA>Av<A>^A<v<A>^AA>Av<A>A<AA>^A', '<AA>A<vA>^A<v<A>^AA>Av<A>A<AA>^A', '<AA>Av<A^>A<v<A>^AA>Av<A>A<AA>^A', '<AA>A<vA^>A<v<A>^AA>Av<A>A<AA>^A', '<AA>Av<A>^Av<<A>^AA>A<vA>A<AA>^A', '<AA>A<vA>^Av<<A>^AA>A<vA>A<AA>^A', '<AA>Av<A^>Av<<A>^AA>A<vA>A<AA>^A', '<AA>A<vA^>Av<<A>^AA>A<vA>A<AA>^A', '<AA>Av<A>^A<v<A>^AA>A<vA>A<AA>^A', '<AA>A<vA>^A<v<A>^AA>A<vA>A<AA>^A', '<AA>Av<A^>A<v<A>^AA>A<vA>A<AA>^A', '<AA>A<vA^>A<v<A>

In [51]:
# CHEATING:

# https://adventofcode.com/2024/day/21#part2
# This is a more efficient and concise solution to part 1 as well, with the appropriate modification to the 'next_robot' procedure

numpad = {'7': (0, 0), '8': (1, 0), '9': (2, 0), '4': (0, 1), '5': (1, 1), '6': (2, 1), '1': (0, 2), '2': (1, 2), '3': (2, 2), '0': (1, 3), 'A': (2, 3)}
dirpad = {'^': (1, 0), 'A': (2, 0), '<': (0, 1), 'v': (1, 1), '>': (2, 1)}
keypad_dirs = {}
for num1, loc1 in numpad.items():
    for num2, loc2 in numpad.items():
        num_pair = (num1, num2)
        if loc1[0] == loc2[0]: keypad_dirs[num_pair] = ('v' if loc1[1] < loc2[1] else '^') * abs(loc1[1] - loc2[1])
        elif loc1[1] == loc2[1]: keypad_dirs[num_pair] = ('>' if loc1[0] < loc2[0] else '<') * abs(loc1[0] - loc2[0])
        else:
            if loc1[0] == 0 and loc2[1] == 3: keypad_dirs[num_pair] = '>' * (loc2[0] - loc1[0]) + 'v' * (loc2[1] - loc1[1])
            elif loc1[1] == 3 and loc2[0] == 0: keypad_dirs[num_pair] = '^' * (loc1[1] - loc2[1]) + '<' * (loc1[0] - loc2[0])
            else: keypad_dirs[num_pair] = [('v' if loc1[1] < loc2[1] else '^') * abs(loc1[1] - loc2[1]), ('>' if loc1[0] < loc2[0] else '<') * abs(loc1[0] - loc2[0])]
for dir1, loc1 in dirpad.items():
    for dir2, loc2 in dirpad.items():
        dir_pair = (dir1, dir2)
        if loc1[0] == loc2[0]: keypad_dirs[dir_pair] = ('v' if loc1[1] < loc2[1] else '^') * abs(loc1[1] - loc2[1])
        elif loc1[1] == loc2[1]: keypad_dirs[dir_pair] = ('>' if loc1[0] < loc2[0] else '<') * abs(loc1[0] - loc2[0])
        else:
            if loc1[0] == 0 and loc2[1] == 0: keypad_dirs[dir_pair] = '>' * (loc2[0] - loc1[0]) + '^'
            elif loc1[1] == 0 and loc2[0] == 0: keypad_dirs[dir_pair] = 'v' + (loc1[0] - loc2[0]) * '<'
            else: keypad_dirs[dir_pair] = [('v' if loc1[1] < loc2[1] else '^') * abs(loc1[1] - loc2[1]), ('>' if loc1[0] < loc2[0] else '<') * abs(loc1[0] - loc2[0])]
known_sequences = {}

def next_robot(new_sequence, level):
    if (new_sequence, level) in known_sequences: return known_sequences[(new_sequence, level)]
    if level == 26:
    #for part 1, just change '26' to '3'
        n = len(new_sequence)
    else:
        n = 0
        for i, c in enumerate(new_sequence):
            presses = keypad_dirs[('A' if i == 0 else new_sequence[i - 1], c)]
            n += min(next_robot(presses[0] + presses[1] + 'A', level + 1), next_robot(presses[1] + presses[0] + 'A', level + 1)) if isinstance(presses, list) else next_robot(presses + 'A', level + 1)
    known_sequences[(new_sequence, level)] = n
    return n

print(sum([next_robot(code, 0) * int(code[:-1]) for code in data_to_use]))

293919502998014


In [52]:
keypad_dirs

{('7', '7'): '',
 ('7', '8'): '>',
 ('7', '9'): '>>',
 ('7', '4'): 'v',
 ('7', '5'): ['v', '>'],
 ('7', '6'): ['v', '>>'],
 ('7', '1'): 'vv',
 ('7', '2'): ['vv', '>'],
 ('7', '3'): ['vv', '>>'],
 ('7', '0'): '>vvv',
 ('7', 'A'): '>>vvv',
 ('8', '7'): '<',
 ('8', '8'): '',
 ('8', '9'): '>',
 ('8', '4'): ['v', '<'],
 ('8', '5'): 'v',
 ('8', '6'): ['v', '>'],
 ('8', '1'): ['vv', '<'],
 ('8', '2'): 'vv',
 ('8', '3'): ['vv', '>'],
 ('8', '0'): 'vvv',
 ('8', 'A'): ['vvv', '>'],
 ('9', '7'): '<<',
 ('9', '8'): '<',
 ('9', '9'): '',
 ('9', '4'): ['v', '<<'],
 ('9', '5'): ['v', '<'],
 ('9', '6'): 'v',
 ('9', '1'): ['vv', '<<'],
 ('9', '2'): ['vv', '<'],
 ('9', '3'): 'vv',
 ('9', '0'): ['vvv', '<'],
 ('9', 'A'): 'vvv',
 ('4', '7'): '^',
 ('4', '8'): ['^', '>'],
 ('4', '9'): ['^', '>>'],
 ('4', '4'): '',
 ('4', '5'): '>',
 ('4', '6'): '>>',
 ('4', '1'): 'v',
 ('4', '2'): ['v', '>'],
 ('4', '3'): ['v', '>>'],
 ('4', '0'): '>vv',
 ('4', 'A'): '>>vv',
 ('5', '7'): ['^', '<'],
 ('5', '8'): '^',
 ('5'